In [9]:
%matplotlib inline

import pandas as pd
import numpy as np
import sys
import os
from tqdm import tqdm
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
import warnings
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 10)
# warnings.filterwarnings('ignore')
def group_feature(df, key, target, aggs):   
    agg_dict = {}
    for ag in aggs:
        agg_dict[f'{target}_{ag}'] = ag
    print(agg_dict)
    t = df.groupby(key)[target].agg(agg_dict).reset_index()
    return t

def extract_feature(df, train_feature):
    t = group_feature(df, 'ship','x',['max','min','mean','std','skew','sum'])
    train_feature = pd.merge(train_feature, t, on='ship', how='left')
    t = group_feature(df, 'ship','x',['count'])
    train_feature = pd.merge(train_feature, t, on='ship', how='left')
    t = group_feature(df, 'ship','y',['max','min','mean','std','skew','sum'])
    train_feature = pd.merge(train_feature, t, on='ship', how='left')
    t = group_feature(df, 'ship','v',['max','min','mean','std','skew','sum'])
    train_feature = pd.merge(train_feature, t, on='ship', how='left')
    t = group_feature(df, 'ship','d',['max','min','mean','std','skew','sum'])
    train_feature = pd.merge(train_feature, t, on='ship', how='left')
    train_feature['x_max_x_min'] = train_feature['x_max'] - train_feature['x_min']
    train_feature['y_max_y_min'] = train_feature['y_max'] - train_feature['y_min']
    train_feature['y_max_x_min'] = train_feature['y_max'] - train_feature['x_min']
    train_feature['x_max_y_min'] = train_feature['x_max'] - train_feature['y_min']
    train_feature['slope'] = train_feature['y_max_y_min'] / np.where(train_feature['x_max_x_min']==0, 0.001, train_feature['x_max_x_min'])
    train_feature['area'] = train_feature['x_max_x_min'] * train_feature['y_max_y_min']
    
    mode_hour = df.groupby('ship')['hour'].agg(lambda x:x.value_counts().index[0]).to_dict()
    train_feature['mode_hour'] = train_feature['ship'].map(mode_hour)
    
    t = group_feature(df, 'ship','hour',['max','min'])
    train_feature = pd.merge(train_feature, t, on='ship', how='left')
    
    hour_nunique = df.groupby('ship')['hour'].nunique().to_dict()
    date_nunique = df.groupby('ship')['date'].nunique().to_dict()
    train_feature['hour_nunique'] = train_feature['ship'].map(hour_nunique)
    train_feature['date_nunique'] = train_feature['ship'].map(date_nunique)

    t = df.groupby('ship')['time'].agg({'diff_time':lambda x:np.max(x)-np.min(x)}).reset_index()
    t['diff_day'] = t['diff_time'].dt.days
    t['diff_second'] = t['diff_time'].dt.seconds
    train_feature = pd.merge(train_feature, t, on='ship', how='left')
    return train_feature

#this func try to sparse the TIme data
def extract_dt(df):
    df['time'] = pd.to_datetime(df['time'], format='%m%d %H:%M:%S')
    # df['month'] = df['time'].dt.month
    # df['day'] = df['time'].dt.day
    df['date'] = df['time'].dt.date
    df['hour'] = df['time'].dt.hour
    # df = df.drop_duplicates(['ship','month'])
    df['weekday'] = df['time'].dt.weekday
    return df

In [10]:
#add FFT
def fft_v(groupbied):
    nSampleNum = len(groupbied)
    sampleTime = groupbied['diff_time'].mean()
    ncount = (nSampleNum-1)*sampleTime
    delta_f = nSampleNum / ncount
    x = np.linspace(0,sampleTime,ncount)#时域波形x轴坐标
    freqLine = nSampleNum
    v = groupbied['v'].values#np.sin(2*pi*250*x)
    fft = abs(np.fft.fft(v))[0:freqLine]  #调用fft变换算法计算频域波形
    fftx = np.linspace(0,delta_f*freqLine,freqLine)  #频域波形x轴坐标
    fft_low = pd.DataFrame(fft[fftx<freqLine*0.25]).describe().T.add_prefix('fft_low_')
    fft_mid_l = pd.DataFrame(fft[(fftx>=freqLine*0.25) & (fftx<freqLine*0.5)]).describe().T.add_prefix('fft_mid_l_')
    fft_mid_h = pd.DataFrame(fft[(fftx>=freqLine*0.5) & (fftx<freqLine*0.75)]).describe().T.add_prefix('fft_mid_h_')
    fft_high = pd.DataFrame(fft[fftx>=freqLine*0.75]).describe().T.add_prefix('fft_high_')
    FFT_=pd.concat([fft_low, fft_mid_l, fft_mid_h, fft_high], axis=1)
    return FFT_

In [11]:
def feature_version2(df,feature_label):
    df_order=df.groupby('ship').apply(lambda x: x.sort_values('time',ascending=True))
    df_order.reset_index(level=0,drop=True,inplace=True)# save the inner index number step#1
    df_order.index.name ='inner_rev_index'# save the inner index number step#2
    df_order.reset_index(drop=False,inplace=True)#add 0-N index
    df_order['inner_ord_index']=df.index#use the former subIndex to Mark the t0-tN
    df_order['diff_time']=pd.DataFrame(df_order.groupby('ship').apply(lambda x:x['time'].diff()/np.timedelta64(1, 'h'))).reset_index()['time']
    df_order['diff_dist']=pd.DataFrame(df_order.groupby('ship').apply(lambda x:np.sqrt(x['x'].diff()**2+x['y'].diff()**2))).reset_index()[0]
    df_order['est_v']=pd.DataFrame(df_order.groupby('ship').apply(lambda x:x['diff_dist']/x['diff_time'])).reset_index()[0]
    df_order['est_diff_dist']=pd.DataFrame(df_order.groupby('ship').apply(lambda x:x['v']*x['diff_time']*1852)).reset_index()[0]
    v_feature=pd.DataFrame(df_order.groupby('ship').apply(fft_v)).reset_index()
    feature_label = pd.merge(feature_label, v_feature, on='ship', how='left')
    temp=pd.DataFrame(df_order[df_order['inner_ord_index']==0][['ship','x','y']]).reset_index(drop=True).rename(columns={'x':'start_x', 'y':'start_y'}, inplace = False)
    feature_label = pd.merge(feature_label, temp, on='ship', how='left')
    temp=pd.DataFrame(df_order[df_order['inner_rev_index']==0][['ship','x','y']]).reset_index(drop=True).rename(columns={'x':'end_x', 'y':'end_y'}, inplace = False)
    feature_label = pd.merge(feature_label, temp, on='ship', how='left')
    return feature_label

In [12]:
train = pd.read_hdf('../input/train.h5')
test = pd.read_hdf('../input/test.h5')

In [13]:
#transform the ori data to col=[ship x y v d time type date hour weekday]
train = extract_dt(train)
test = extract_dt(test)

In [14]:
#drop out and only leave the default 1st time data like the beginning time and position
train_label = train.drop_duplicates('ship')
test_label = test.drop_duplicates('ship')

In [15]:
#zip realize the one to one map from the former list to the latter list
#dict make the map as a dictionary
type_map = dict(zip(train_label['type'].unique(), np.arange(3)))
# rev means a reversed map relation
type_map_rev = {v:k for k,v in type_map.items()} 
train_label['type'] = train_label['type'].map(type_map)
#the data of type was now described by numbers as 0,1,2 

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [16]:
train_feature = extract_feature(train, train_label)
train_feature = feature_version2(train, train_feature)

{'x_max': 'max', 'x_min': 'min', 'x_mean': 'mean', 'x_std': 'std', 'x_skew': 'skew', 'x_sum': 'sum'}
{'x_count': 'count'}
{'y_max': 'max', 'y_min': 'min', 'y_mean': 'mean', 'y_std': 'std', 'y_skew': 'skew', 'y_sum': 'sum'}
{'v_max': 'max', 'v_min': 'min', 'v_mean': 'mean', 'v_std': 'std', 'v_skew': 'skew', 'v_sum': 'sum'}
{'d_max': 'max', 'd_min': 'min', 'd_mean': 'mean', 'd_std': 'std', 'd_skew': 'skew', 'd_sum': 'sum'}
{'hour_max': 'max', 'hour_min': 'min'}


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:54: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  


In [17]:
# train_feature = pd.merge(train_feature, v_feature, on='ship', how='left')

In [18]:
test_label = extract_feature(test, test_label)
test_label = feature_version2(test, test_label)

{'x_max': 'max', 'x_min': 'min', 'x_mean': 'mean', 'x_std': 'std', 'x_skew': 'skew', 'x_sum': 'sum'}
{'x_count': 'count'}
{'y_max': 'max', 'y_min': 'min', 'y_mean': 'mean', 'y_std': 'std', 'y_skew': 'skew', 'y_sum': 'sum'}
{'v_max': 'max', 'v_min': 'min', 'v_mean': 'mean', 'v_std': 'std', 'v_skew': 'skew', 'v_sum': 'sum'}
{'d_max': 'max', 'd_min': 'min', 'd_mean': 'mean', 'd_std': 'std', 'd_skew': 'skew', 'd_sum': 'sum'}
{'hour_max': 'max', 'hour_min': 'min'}


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:54: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  


In [19]:
features = [x for x in train_feature.columns if x not in ['ship','type','time','diff_time','date']]
target = 'type'

In [20]:
print(len(features), ','.join(features))

81 x,y,v,d,hour,weekday,x_max,x_min,x_mean,x_std,x_skew,x_sum,x_count,y_max,y_min,y_mean,y_std,y_skew,y_sum,v_max,v_min,v_mean,v_std,v_skew,v_sum,d_max,d_min,d_mean,d_std,d_skew,d_sum,x_max_x_min,y_max_y_min,y_max_x_min,x_max_y_min,slope,area,mode_hour,hour_max,hour_min,hour_nunique,date_nunique,diff_day,diff_second,level_1,fft_low_count,fft_low_mean,fft_low_std,fft_low_min,fft_low_25%,fft_low_50%,fft_low_75%,fft_low_max,fft_mid_l_count,fft_mid_l_mean,fft_mid_l_std,fft_mid_l_min,fft_mid_l_25%,fft_mid_l_50%,fft_mid_l_75%,fft_mid_l_max,fft_mid_h_count,fft_mid_h_mean,fft_mid_h_std,fft_mid_h_min,fft_mid_h_25%,fft_mid_h_50%,fft_mid_h_75%,fft_mid_h_max,fft_high_count,fft_high_mean,fft_high_std,fft_high_min,fft_high_25%,fft_high_50%,fft_high_75%,fft_high_max,start_x,start_y,end_x,end_y


In [21]:
params = {
    'num_leaves': 2 ** 10,
    'learning_rate': 0.005,
    'min_child_samples': 20,
    'boosting': 'gbdt',
    'objective': 'multiclass',
    'n_estimators': 20000,
    'metric': 'multi_logloss',
    'num_class': 3,
    'feature_fraction': .75,
    'bagging_fraction': .75,
    'seed': 99,
    'num_threads': 8,
    'verbose': -1
    }
    # 'task': 'train',
    # 'boosting_type': 'gbdt',  # 设置提升类型
    # 'objective': 'regression', # 目标函数
    # 'metric': {'l2', 'auc'},  # 评估函数
    # 'num_leaves': 31,   # 叶子节点数
    # 'learning_rate': 0.05,  # 学习速率
    # 'feature_fraction': 0.9, # 建树的特征选择比例
    # 'bagging_fraction': 0.8, # 建树的样本采样比例
    # 'bagging_freq': 5,  # k 意味着每 k 次迭代执行bagging
    # 'verbose': 1 # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
# params = {
#     'n_estimators': 10000,
#     'boosting_type': 'gbdt',
#     'objective': 'multiclass',
#     'num_class': 3,
#     'early_stopping_rounds': 100,
# }

In [22]:
fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=74)

X = train_feature[features].copy()
y = train_feature[target]
models = []
pred = np.zeros((len(test_label),3))
oof = np.zeros((len(X), 3))
for index, (train_idx, val_idx) in enumerate(fold.split(X, y)):

    train_set = lgb.Dataset(X.iloc[train_idx], y.iloc[train_idx])
    val_set = lgb.Dataset(X.iloc[val_idx], y.iloc[val_idx])

    model = lgb.train(params, train_set, valid_sets=[train_set, val_set], verbose_eval=100)
    models.append(model)
    val_pred = model.predict(X.iloc[val_idx])
    oof[val_idx] = val_pred
    val_y = y.iloc[val_idx]
    val_pred = np.argmax(val_pred, axis=1)
    print(index, 'val f1', metrics.f1_score(val_y, val_pred, average='macro'))
    # 0.8695539641133697
    # 0.8866211724839532

    test_pred = model.predict(test_label[features])
    pred += test_pred/5
#???why not 10? pred += test_pred/10 #as fold=10

D:\Anaconda\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[100]	training's multi_logloss: 0.571703	valid_1's multi_logloss: 0.618591
[200]	training's multi_logloss: 0.392153	valid_1's multi_logloss: 0.471521
[300]	training's multi_logloss: 0.281569	valid_1's multi_logloss: 0.386542
[400]	training's multi_logloss: 0.207532	valid_1's multi_logloss: 0.333805
[500]	training's multi_logloss: 0.155288	valid_1's multi_logloss: 0.298932
[600]	training's multi_logloss: 0.117507	valid_1's multi_logloss: 0.275849
[700]	training's multi_logloss: 0.089485	valid_1's multi_logloss: 0.259644
[800]	training's multi_logloss: 0.0686047	valid_1's multi_logloss: 0.249214
[900]	training's multi_logloss: 0.0528668	valid_1's multi_logloss: 0.241429
[1000]	training's multi_logloss: 0.040851	valid_1's multi_logloss: 0.234843
[1100]	training's multi_logloss: 0.0316079	valid_1's multi_logloss: 0.230794
[1200]	training's multi_logloss: 0.0244925	valid_1's multi_logloss: 0.227587
[1300]	training's multi_logloss: 0.0189876	valid_1's multi_logloss: 0.226662
[1400]	training'

In [23]:
oof = np.argmax(oof, axis=1)
print('oof f1', metrics.f1_score(oof, y, average='macro'))
# 0.8658824154822908

oof f1 0.8910536096427008


In [24]:
#因为是找最大的，所以平均与否区别不大
pred = np.argmax(pred, axis=1)
sub = test_label[['ship']]
sub['pred'] = pred

print(sub['pred'].value_counts(1))
sub['pred'] = sub['pred'].map(type_map_rev)
sub.to_csv('result.csv', index=None, header=None)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


0    0.6350
1    0.2345
2    0.1305
Name: pred, dtype: float64


In [25]:
ret = []
for index, model in enumerate(models):
    df = pd.DataFrame()
    df['name'] = model.feature_name()
    df['score'] = model.feature_importance()
    df['fold'] = index
    ret.append(df)
    
df = pd.concat(ret)

In [26]:
df = df.groupby('name', as_index=False)['score'].mean()
df = df.sort_values(['score'], ascending=False)

In [27]:
df

,name,score
74,y_max_x_min,116464.3
66,x_max_y_min,109276.6
53,start_y,91646.3
52,start_x,90070.9
58,v_skew,89802.6
68,x_min,84564.0
78,y_skew,80847.8
73,y_max,80765.6
69,x_skew,78714.3
51,slope,76188.3


In [29]:
#save the models
for index, model in enumerate(models):
    save_base_path='../model/'
    name = str(index)
    save_path = os.path.join(save_base_path, name + '.txt')
    model.save_model(save_path)
    
print('model saved')

model saved


In [ ]:
# model1 = lgb.train(params, train_set, valid_sets=[train_set, val_set], verbose_eval=100)
# model1.load_model(save_path)

D:\Anaconda\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[100]	training's multi_logloss: 0.573467	valid_1's multi_logloss: 0.610899
